In [1]:
import cmdstanpy
import numpy as np
import pandas as pd

from IPython.display import clear_output
from itertools import product
cmdstanpy.install_cmdstan();

CmdStan install directory: /Users/igor.michels/.cmdstan
CmdStan version 2.34.1 already installed
Test model compilation


In [2]:
NUM_SEASONS = 1
NUM_TEAMS = 20

In [3]:
def generate_matchups(n_equipes, habilidades, home_force):
    if home_force is None: home_force = 0
    confrontos = []
    for equipe1, equipe2 in product(range(1, n_equipes + 1), repeat=2):
        if equipe1 == equipe2: continue
        prob = np.exp(habilidades[equipe1 - 1] + home_force)
        prob = prob / (np.exp(habilidades[equipe1 - 1] + home_force) + np.exp(habilidades[equipe2 - 1]))
        confrontos.append({
            'equipe1': equipe1,
            'equipe2': equipe2,
            'vitoria_equipe1': (np.random.random() < prob) * 1
        })

    df_confrontos = pd.DataFrame(confrontos)

    return df_confrontos

def generate_seasons(n_equipes, habilidades, n_seasons, home_force=None):
    df = pd.DataFrame()
    for _ in range(n_seasons):
        df = pd.concat([df, generate_matchups(n_equipes, habilidades, home_force)])
    
    return df

In [4]:
habilidades = np.random.normal(0, 1, NUM_TEAMS)
df = generate_seasons(NUM_TEAMS, habilidades, NUM_SEASONS)

modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/BT_model.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': NUM_TEAMS,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'vitoria_equipe1': df['vitoria_equipe1'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
habilidades += ajuste.summary()['Mean']['habilidade[1]'] - habilidades[0]
inside_ci = (habilidades > ajuste.summary()['5%'].values[1:])
inside_ci *= ((habilidades < ajuste.summary()['95%'].values[1:]))

clear_output()
sum(inside_ci) / NUM_TEAMS

0.8

In [5]:
home_force = abs(np.random.normal(0, 1))
habilidades = np.random.normal(0, 1, NUM_TEAMS)
df = generate_seasons(NUM_TEAMS, habilidades, NUM_SEASONS, home_force)

modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/BT_model_2.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': NUM_TEAMS,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'vitoria_equipe1': df['vitoria_equipe1'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
habilidades += ajuste.summary()['Mean']['habilidade[1]'] - habilidades[0]
inside_ci = (habilidades > ajuste.summary()['5%'].values[2:])
inside_ci *= ((habilidades < ajuste.summary()['95%'].values[2:]))

clear_output()
sum(inside_ci) / NUM_TEAMS

0.55

In [6]:
home_force

0.6668953786312211

In [7]:
ajuste.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,1.021170e+07,1.024090e+06,3.468070e+06,3.235220e+06,1.086110e+07,1.455060e+07,11.4682,1.633190,1.735770e+00
home_force,9.777520e+00,1.350320e-01,4.729970e-01,8.702020e+00,9.903910e+00,1.026700e+01,12.2700,1.747370,1.489170e+00
habilidade[1],7.415910e-01,9.905690e-01,1.401090e+00,-6.155990e-01,9.867150e-01,2.930350e+00,2.0006,0.284905,7.949520e+05
habilidade[2],4.349040e-01,6.619770e-01,9.363180e-01,-4.735130e-01,4.554960e-01,1.948140e+00,2.0006,0.284905,9.703540e+05
habilidade[3],9.702990e-01,6.083880e-01,8.605200e-01,-4.762930e-01,1.350030e+00,1.783000e+00,2.0006,0.284905,6.592830e+05
habilidade[4],3.310380e-01,5.697530e-01,8.058730e-01,-4.286550e-01,2.761840e-01,1.656400e+00,2.0006,0.284905,9.784590e+05
habilidade[5],1.058360e+00,4.667680e-01,6.602080e-01,3.787510e-01,1.173320e+00,2.085310e+00,2.0006,0.284905,8.731390e+05
habilidade[6],-3.263210e-01,5.513890e-01,7.798980e-01,-1.187770e+00,1.209260e-01,7.189310e-01,2.0006,0.284905,1.121008e+06
habilidade[7],5.319850e-01,7.261280e-01,1.027050e+00,-4.756990e-01,1.091340e+00,1.937340e+00,2.0006,0.284905,6.366260e+05
habilidade[8],7.053070e-01,4.561320e-01,6.451650e-01,-3.088420e-01,9.493210e-01,1.464750e+00,2.0006,0.284905,6.033780e+05
